# Ch 7 Hypothesis and Inference


## binomial dist be approximated to normal dist

* proof by CLT
![](./images/ch07_hypothesis/1.png)

![](./images/ch07_hypothesis/2.png)

![](./images/ch07_hypothesis/3.png)

* proof without CLT
  * http://scipp.ucsc.edu/~haber/ph116C/NormalApprox.pdf
  

## Level of Significance = p-value (type-2 error)

* prob of observing counter-example given null hypothesis is true
* reject null hypothesis even though it is true
* false-positive



## Power of test (type-1 error)
* less formally, probability of rejecting the null when it is not correct
* formally, prob of reject the null when alternative is true

![](./images/ch07_hypothesis/4.png)

## Confidence Interval

* How confident can we be about the estimation 

![](./images/ch07_hypothesis/5.png)
![](./images/ch07_hypothesis/6.png)

# P value hacking, Data dredging


![](./images/ch07_hypothesis/7.png)
![](./images/ch07_hypothesis/8.png)

# A/B Test

* http://statisticalconcepts.blogspot.kr/2010/03/ab-testing.html

![](./images/ch07_hypothesis/9.png)
![](./images/ch07_hypothesis/10.png)
![](./images/ch07_hypothesis/11.png)


# popular distribution

## Gamma

![](./images/ch07_hypothesis/12.png)
![](./images/ch07_hypothesis/13.png)

## Beta

![](./images/ch07_hypothesis/14.png)
![](./images/ch07_hypothesis/15.png)




# Baysian inference 

## Beta-binomial model : prior beta, likelihood binomial

![](./images/ch07_hypothesis/16.png)
![](./images/ch07_hypothesis/17.png)


## Conjugate prior

![](./images/ch07_hypothesis/18.png)






In [28]:
import math

def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    low_z, low_p = -10.0, 0 # normal_cdf(-10) is (very close to) 0
    hi_z, hi_p = 10.0, 1 # normal_cdf(10) is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2 # consider the midpoint
        mid_p = normal_cdf(mid_z) # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break
        return mid_z

def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma


# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [29]:
# let’s say that we choose to flip the coin n = 1000 times. If our hypothesis of fairness is true,
normal_approximation_to_binomial(1000, 0.5)


(500.0, 15.811388300841896)

In [30]:
# significance, 0.5
normal_two_sided_bounds(0.95, mu_0, sigma_0)


(500.0, 500.0)

In [31]:
# power of test, two-sided

# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print lo, hi

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
print mu_1, sigma_1

# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability # 0.887
print type_2_probability
print power



500.0 500.0
550.0 15.7321327226
0.0
1.0


In [32]:
# power of test, one-sided

hi = normal_upper_bound(0.95, mu_0, sigma_0)
print hi

# is 526 (< 531, since we need more probability in the upper tail)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability # 0.936

print type_2_probability
print power

500.0
0.00074094038736
0.999259059613


In [33]:
# p-value, in two-sided test

def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)
    
#  If we were to see 530 heads, we would compute:
two_sided_p_value(529.5, mu_0, sigma_0) # 0.062

0.06207721579598857

In [34]:
import random
from __future__ import division

# extreme count

extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000)) # in 1000 flips
    if num_heads >= 530 or num_heads <= 470: # and count how often
        extreme_value_count += 1 # the # is 'extreme'

print extreme_value_count / 100000 # 0.062

0.06155


In [35]:
# p-hacking

def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])
print num_rejections # 46

46
